---
title: "Logging - How to and best practices"
author: "Damien Martin"
date: "2024-10-17 12:00"
categories: [python, logging]
description: "Giving some of the best logging practices, and clarifying some of the logging confusion."
---

# Problem

Logging in Python is very flexible, but has some odd defaults that make it non-intuitive to use. 

The simplest example is the following code, which goes not print out anything:

In [1]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # Should print at level logging.INFO and above

logger.info("Hello")

See? Nothing printed. We can try the following solution:

In [2]:
import logging

logger = logging.getLogger('example')
logger.setLevel(logging.INFO)
logger.info('one')   # nothing printed
logging.info('two')  # nothing printed
logger.info('three') # prints / logs "three"

INFO:example:three


Why is only three printed? We will see why after looking at the next example, which fixes these problems.

In [7]:
import logging

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Should print at level logging.DEBUG and above

logger.info("Hello")

INFO:__main__:Hello


The addition of `logging.basicConfig()` now allows you to set the level in a way that you expect! In our previous example, where we tried to log `one`, `two`, and `three`, the call to `logging.info` implicitly called the `logging.basicConfig()`, which then affects the all loggers from here on out. This call is changing _the root logging handler_, something we will define later on.

To show that this is a change that persists, let's run the same code sample in the same kernel:

In [8]:
#| lst-cap: Same code as above, but now `logging.basicConfig()` has been called
import logging

logger = logging.getLogger('example')
logger.setLevel(logging.INFO)
logger.info('one')   # nothing printed
logging.info('two')  # nothing printed
logger.info('three') # prints / logs "three"

INFO:example:one
INFO:example:three


# Solutions in a hurry

## I just want something that works for simple apps

If you just want to get logging, the basic solution is 

1. Call `logging.basicConfig()` early. This means doing it before other modules!

The problems with this approach are:
1. The call to `logging.basicConfig()` affects the root logger, so all loggigng is affected. This causes noisy libraries to propogate their error messages.
2. The call to `logging.basicConfig()` is only processed once, so you have to make sure it isn't called by any of your imports. This also means that your import is going to fight with everyone else's. It isnt' a problem if you are all trying to set the level to DEBUG in the basic config.


Calling `logging.basicConfig()` acts on the root logger, and because logging is handled in a hierarchy (with the root logger at the root), this can make it hard to reason about. There is only one root logger in your application, so you have to be aware of these side affects for not only everything you use, but everything you import.

This [stackoverflow answer](https://stackoverflow.com/questions/57115395/why-does-logger-info-only-appear-after-calling-logging-info/) discusses this in more detail.


## The better approach

# Understanding Logging